# Module Attendance

This module attendance program displays the weekly attendance of a module the user or tutor chooses to view. It shows a dataframe of each module session per week by performing the following functions:
- reads module attendance and attendance details from the database
- merges the module attendance and attendance details into a dataframe
- calculates the attendance percentage of each module session
- formats the dataframe based on a condition

## Imports

In [21]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

## Functions

### Extracting Information from Database

In [22]:
def get_session_df_from_db(dfsession, week_no):
    '''
    extracting specific columns of sessions table by week from the database
    input:
        dfsession:the session table name
        week_no: the week number to filter table by, string type
    return: 
        dfSess: the filtered attendance details (sessions) dataframe
    '''
    conn = sqlite3.connect('C:\\Users\\rolli\\CW\\cop504cwdata\\CWDatabase.db')
    
    dfSess=pd.read_sql('''SELECT A.start_time, A.room, A.type \
                           FROM %s as A WHERE week == '%s';'''\
                           %(dfsession, week_no),con=conn)
    dfSess.index.names=["id"]
    conn.close()
    
    return dfSess

def session_per_week_df_from_db(dfsessions, dfMod, week_no):
    '''
    extracting the joined sessions and module attendance tables from database
    output:
        returns a dataframe of each students attendance score across the module sessions per week
    '''
    conn = sqlite3.connect('C:\\Users\\rolli\\CW\\cop504cwdata\\CWDatabase.db')
    
    dfMod1=pd.read_sql('''SELECT A.start_time, A.room, A.type, B.*\
                           FROM %s A JOIN %s B ON A.sessions_id=B.sessions_id\
                           WHERE week == '%s';'''%(dfsessions, dfMod, week_no),con=conn)
    dfMod1=dfMod1.reset_index()
    conn.close()
    
    return dfMod1


### Calculating and Merging Dataframes

In [23]:
def module_attendance_percentage(attendance_score):
    '''
    calculating students attendance percentage for each module session
    '''
    score_1=attendance_score.iloc[:,5:].count(axis=1)
    score_2=attendance_score.iloc[:,5:].sum(axis=1)
    score_percentage=(score_2/score_1)*100
    dfS=score_percentage.to_frame()
    dfS.columns=["Attendance%"]
    dfS.index.names=["id"]
    
    return dfS

def module_attendance_record(attendance_df, dfsess):
    '''
    merges attendance percentage dataframe to attendance details(sessions) dataframe
    output:
        returns weekly module attendance, dataframe 
    '''
    reg = pd.merge(attendance_df,dfsess, how="inner", on="id")
    reg = reg.rename(columns = {"start_time":"time"})
    
    return reg
    

### Output

In [24]:
def acolor_code(value):
    """
    Setting background colours 
    of table cells 
    value is a cell value. 
    """
    
    try:
        if value is None or math.isnan(value):
            return None
        if value < 40:
            colour = 'red'
        elif value < 70:
            colour = 'yellow'
        else:
            colour = 'green'
        
    except:
        print('value:', value)
        assert False
        
    styler=f'background-color: {colour}'
    return styler


def format_output(dfData):
    """
    applying formatting function to the dataframe.
    """
    styledOutput=dfData.style.applymap(acolor_code, subset=['Attendance%'])\
                              .format(precision=1)
    return styledOutput

## Main Code

In [25]:
def module_attendance(sessions,module, week):
    '''
    main function of the module attendance code 
    '''
    sessions = module+"_Sessions"
    weekly_mod_sessions=session_per_week_df_from_db(sessions, module, week)
    weekly_mod_sessions1=module_attendance_percentage(weekly_mod_sessions)
    sessions=get_session_df_from_db(sessions, week)
    mod_att_record1=module_attendance_record(sessions,weekly_mod_sessions1)
    format_input=format_output(mod_att_record1)

    return format_input

## Test

In [26]:
# weekly module attendance session
weekly_mod_sessions=session_per_week_df_from_db('COA111_Sessions', 'COA111', '1')
weekly_mod_sessions

,index,start_time,room,type,sessions_id,0,1,2,3,4,...,233,234,235,236,237,238,239,240,241,242
0,0,09:00,U020,Lecture,0,1,1,1,None,1,...,1.0,1,1,1,1,0,1,1,1,0
1,1,10:00,SMB014,Lecture,1,1,1,1,None,0,...,NaN,1,1,1,0,0,0,1,1,0


In [27]:
# weekly module attendance percentage
weekly_mod_sessions1=module_attendance_percentage(weekly_mod_sessions)
weekly_mod_sessions1

,Attendance%
id,
0,73.333333
1,65.625000


In [28]:
# attendance details
sessions=get_session_df_from_db('COA111_Sessions', '1')
sessions

,start_time,room,type
id,,,
0,09:00,U020,Lecture
1,10:00,SMB014,Lecture


In [29]:
# weekly module attendance
mod_att_record1=module_attendance_record(sessions,weekly_mod_sessions1)
mod_att_record1

,time,room,type,Attendance%
id,,,,
0,09:00,U020,Lecture,73.333333
1,10:00,SMB014,Lecture,65.625000


In [30]:
# conditional formatting
format_input=format_output(mod_att_record1)
format_input

,time,room,type,Attendance%
id,,,,
0,09:00,U020,Lecture,73.3
1,10:00,SMB014,Lecture,65.6
